In [1]:
# Let's see how to use DBT to:
# 1: train a VGG-like network on CIFAR-10
# 2: continue a train from the last iteration
# 3: do TRANSFER LEARNING from the trained model to another model that will be able to classify CIFAR-100
# 4: do FINE TUNING of the model trained on CIFAR-10 to solve the CIFAR-100 classification problem
# 5: compare the train/validation/test performance of the models

import pandas as pd
import pprint
import tensorflow as tf
from dytb.inputs.predefined import Cifar10, Cifar100
from dytb.train import train
from dytb.models.predefined.VGG import VGG

In [2]:
# Instantiate the model
vgg = VGG()

In [3]:
# Instantiate the CIFAR-10 input source
cifar10 = Cifar10.Cifar10()

In [4]:
# 1: Train VGG on Cifar10 for an Epoch

# Place the train process on GPU:0
device = '/gpu:0'
with tf.device(device):
    info = train(
        model=vgg,
        dataset=cifar10,
        hyperparameters={
            "epochs": 1,
            "batch_size": 50,
            "regularizations": {
                "l2": 1e-5,
                "augmentation": {
                    "name": "FlipLR",
                    "fn": tf.image.random_flip_left_right,
                    # On average the training set size double appling this
                    # transformation, thus factor=2
                    "factor": 2,
                }
            },
            "gd": {
                "optimizer": tf.train.AdamOptimizer,
                "args": {
                    "learning_rate": 1e-3,
                    "beta1": 0.9,
                    "beta2": 0.99,
                    "epsilon": 1e-8
                }
            }
        },
        force_restart=True)

Original training set size 50000. Augmented training set size: 100000
<tf.Variable 'VGG/64/conv1/W:0' shape=(3, 3, 3, 64) dtype=float32_ref>
<tf.Variable 'VGG/64/conv1/b:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'VGG/64/conv2/W:0' shape=(3, 3, 64, 64) dtype=float32_ref>
<tf.Variable 'VGG/64/conv2/b:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'VGG/128/conv3/W:0' shape=(3, 3, 64, 128) dtype=float32_ref>
<tf.Variable 'VGG/128/conv3/b:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'VGG/128/conv4/W:0' shape=(3, 3, 128, 128) dtype=float32_ref>
<tf.Variable 'VGG/128/conv4/b:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'VGG/256/conv5/W:0' shape=(3, 3, 128, 256) dtype=float32_ref>
<tf.Variable 'VGG/256/conv5/b:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'VGG/256/conv6/W:0' shape=(3, 3, 256, 256) dtype=float32_ref>
<tf.Variable 'VGG/256/conv6/b:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'VGG/256/conv7/W:0' shape=(3, 3, 256, 256) dtype=float32_ref>
<tf.Variable 'VGG/256/conv7

In [5]:
# Info contains every information related to the trained model.
# We're interested in stats only, thus we extract only them from the info dict
# Display the results in a table. Let's use a Pandas DataFrame for that

# Extract the accuracyes measured in every set (train/validation/test)
accuracies = {key: value["accuracy"] for key, value in info["stats"].items()}
df = pd.DataFrame.from_records(accuracies, index=["accuracy"])
df

,test,train,validation
accuracy,0.6351,0.64872,0.635


In [6]:
# Extract the confusion matrices 
confusion_matrices = {key: value["confusion_matrix"] for key, value in info["stats"].items()}
# Display the confusione matrices for the training set
df = pd.DataFrame(confusion_matrices["train"])
df

,0,1,2,3,4,5,6,7,8,9
0,3258,116,373,55,72,69,21,136,826,123
1,36,4201,8,28,13,6,27,28,121,544
2,383,8,2249,187,841,562,394,166,127,20
3,82,12,510,809,356,2179,658,196,120,105
4,155,8,524,69,2846,257,360,705,32,24
5,29,5,321,317,282,3646,136,225,24,22
6,16,16,157,251,402,150,3822,22,33,120
7,54,0,156,73,290,754,23,3567,18,14
8,324,188,67,43,11,20,31,32,4194,90
9,129,570,15,48,9,53,12,235,120,3859


In [7]:
# 2: train it again for another epoch
# Note the `force_restart` parameter removed.
# `epochs` is the TOTAL number of epoch for the trained model
# Thus since we trained it before for a single epoch,
# we set "epochs": 2 in order to train it for another epoch

with tf.device(device):
    info = train(
        model=vgg,
        dataset=cifar10,
        hyperparameters={
            "epochs": 2,
            "batch_size": 50,
            "regularizations": {
                "l2": 1e-5,
                "augmentation": {
                    "name": "FlipLR",
                    "fn": tf.image.random_flip_left_right,
                    "factor": 2,
                }
            },
            "gd": {
                "optimizer": tf.train.AdamOptimizer,
                "args": {
                    "learning_rate": 1e-3,
                    "beta1": 0.9,
                    "beta2": 0.99,
                    "epsilon": 1e-8
                }
            }
        })

Original training set size 50000. Augmented training set size: 100000
<tf.Variable 'VGG/64/conv1/W:0' shape=(3, 3, 3, 64) dtype=float32_ref>
<tf.Variable 'VGG/64/conv1/b:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'VGG/64/conv2/W:0' shape=(3, 3, 64, 64) dtype=float32_ref>
<tf.Variable 'VGG/64/conv2/b:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'VGG/128/conv3/W:0' shape=(3, 3, 64, 128) dtype=float32_ref>
<tf.Variable 'VGG/128/conv3/b:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'VGG/128/conv4/W:0' shape=(3, 3, 128, 128) dtype=float32_ref>
<tf.Variable 'VGG/128/conv4/b:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'VGG/256/conv5/W:0' shape=(3, 3, 128, 256) dtype=float32_ref>
<tf.Variable 'VGG/256/conv5/b:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'VGG/256/conv6/W:0' shape=(3, 3, 256, 256) dtype=float32_ref>
<tf.Variable 'VGG/256/conv6/b:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'VGG/256/conv7/W:0' shape=(3, 3, 256, 256) dtype=float32_ref>
<tf.Variable 'VGG/256/conv7

In [8]:
# Display the results in a table. Let's use a Pandas DataFrame for that
accuracies = {key: value["accuracy"] for key, value in info["stats"].items()}
df = pd.DataFrame.from_records(accuracies, index=["accuracy"])
df

,test,train,validation
accuracy,0.7314,0.75866,0.7314


In [9]:
# Save last trained model info
vggInfo = info

In [ ]:
# 3: TRANSFER LEARNING
# Use the best model trained on Cifar10, to classify Cifar 100 images.
# Thus we train ONLY the softmax linear scope (that has 100 neurons, now),
# keeping constant any other previosly trained layer
# We load the weights from the previous trained model, or better
# DyTB saves the "best" model (w.r.t. a metric) in a separate folder
# So we extract the info["paths"]["best"] path, that's the path of the best
# model trained so far.
cifar100 = Cifar100.Cifar100()
with tf.device(device):
    transferInfo = train(
        model=vgg,
        dataset=cifar100,
        hyperparameters={
            "epochs": 1,
            "batch_size": 50,
            "regularizations": {
                "l2": 1e-5,
                "augmentation": {
                    "name": "FlipLR",
                    "fn": tf.image.random_flip_left_right,
                    "factor": 2,
                }
            },
            "gd": {
                "optimizer": tf.train.AdamOptimizer,
                "args": {
                    "learning_rate": 1e-3,
                    "beta1": 0.9,
                    "beta2": 0.99,
                    "epsilon": 1e-8
                    }
                }
        },
        force_restart=True,
        surgery={
            "checkpoint_path": vggInfo["paths"]["best"],
            "exclude_scopes": "VGG/softmax_linear",
            "trainable_scopes": "VGG/softmax_linear"
        })

Original training set size 50000. Augmented training set size: 100000
<tf.Variable 'VGG/64/conv1/W:0' shape=(3, 3, 3, 64) dtype=float32_ref>
<tf.Variable 'VGG/64/conv1/b:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'VGG/64/conv2/W:0' shape=(3, 3, 64, 64) dtype=float32_ref>
<tf.Variable 'VGG/64/conv2/b:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'VGG/128/conv3/W:0' shape=(3, 3, 64, 128) dtype=float32_ref>
<tf.Variable 'VGG/128/conv3/b:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'VGG/128/conv4/W:0' shape=(3, 3, 128, 128) dtype=float32_ref>
<tf.Variable 'VGG/128/conv4/b:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'VGG/256/conv5/W:0' shape=(3, 3, 128, 256) dtype=float32_ref>
<tf.Variable 'VGG/256/conv5/b:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'VGG/256/conv6/W:0' shape=(3, 3, 256, 256) dtype=float32_ref>
<tf.Variable 'VGG/256/conv6/b:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'VGG/256/conv7/W:0' shape=(3, 3, 256, 256) dtype=float32_ref>
<tf.Variable 'VGG/256/conv7

In [ ]:
# 4: FINE TUNING:
# Use the model pointed by vggInfo to fine tune the whole network
# and tune it on Cifar100.
# Let's retrain the whole network end-to-end, starting from the learned weights
# Just remove the "traiable_scopes" section from the surgery parameter
with tf.device(device):
    fineTuningInfo = train(
        model=vgg,
        dataset=cifar100,
        hyperparameters={
            "epochs": 1,
            "batch_size": 50,
            "regularizations": {
                "l2": 1e-5,
                "augmentation": {
                    "name": "FlipLR",
                    "fn": tf.image.random_flip_left_right,
                    "factor": 2,
                }
            },
            "gd": {
                "optimizer": tf.train.AdamOptimizer,
                "args": {
                    "learning_rate": 1e-3,
                    "beta1": 0.9,
                    "beta2": 0.99,
                    "epsilon": 1e-8
                }
            }
        },
        force_restart=True,
        surgery={
            "checkpoint_path": vggInfo["paths"]["best"],
            "exclude_scopes": "VGG/softmax_linear"
        })


Original training set size 50000. Augmented training set size: 100000
<tf.Variable 'VGG/64/conv1/W:0' shape=(3, 3, 3, 64) dtype=float32_ref>
<tf.Variable 'VGG/64/conv1/b:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'VGG/64/conv2/W:0' shape=(3, 3, 64, 64) dtype=float32_ref>
<tf.Variable 'VGG/64/conv2/b:0' shape=(64,) dtype=float32_ref>
<tf.Variable 'VGG/128/conv3/W:0' shape=(3, 3, 64, 128) dtype=float32_ref>
<tf.Variable 'VGG/128/conv3/b:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'VGG/128/conv4/W:0' shape=(3, 3, 128, 128) dtype=float32_ref>
<tf.Variable 'VGG/128/conv4/b:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'VGG/256/conv5/W:0' shape=(3, 3, 128, 256) dtype=float32_ref>
<tf.Variable 'VGG/256/conv5/b:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'VGG/256/conv6/W:0' shape=(3, 3, 256, 256) dtype=float32_ref>
<tf.Variable 'VGG/256/conv6/b:0' shape=(256,) dtype=float32_ref>
<tf.Variable 'VGG/256/conv7/W:0' shape=(3, 3, 256, 256) dtype=float32_ref>
<tf.Variable 'VGG/256/conv7

In [ ]:
# Compare the performance of Transfer learning and Fine Tuning
accuracies = {key: value["accuracy"] for key, value in transferInfo["stats"].items()}
df = pd.DataFrame.from_records(accuracies, index=["accuracy"])
df

In [ ]:
accuracies = {key: value["accuracy"] for key, value in fineTuningInfo["stats"].items()}
df = pd.DataFrame.from_records(accuracies, index=["accuracy"])
df

In [ ]:
# For completeness, lets see what a info object contains
pprint.pprint(info, indent=4)